In [16]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.diagnostic as smd
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
import sklearn.metrics as metrics
from sklearn.linear_model import Ridge
# import seaborn as sns
import statsmodels.api as sm
import random
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import LearningCurveDisplay
from sklearn.compose import ColumnTransformer
plt.rcParams['figure.figsize'] = (15, 9)
plt.style.use('ggplot')
random.seed(123)

In [17]:
carros = pd.read_csv("./cars.csv")

***Número de filas:*** 205  
***Número de atributos o columnas:*** 26 

### Información de los atributos:  

|Atributo:|Rango del atributo:|
|--------:|:-------------------:|
|1. symboling:|-3, -2, -1, 0, 1, 2, 3.|
|2. normalized-losses:|numérico 65 hasta 256.|
|3. make:|alfa-romero, audi, bmw, chevrolet, dodge, honda, isuzu, jaguar, mazda, mercedes-benz, mercury, mitsubishi, nissan, peugot, plymouth, porsche,  renault, saab, subaru, hastayota, volkswagen, volvo|
|4. fuel-type:|diesel, gas.|
|5. aspiration:|std, turbo.|
|6. num-of-doors:|four, two.|
|7. body-style:|hardtop, wagon, sedan, hatchback, convertible.|
|8. drive-wheels:|4wd, fwd, rwd.|
|9. engine-location:|front, rear.|
|10. wheel-base:|numérico desde 86.6 hasta 120.9.|
|11. length:|numérico desde 141.1 hasta 208.1.|
|12. width:|numérico desde 60.3 hasta 72.3.|
|13. height:|numérico desde 47.8 hasta 59.8.|
|14. curb-weight:|numérico desde 1488 hasta 4066.|
|15. engine-type:|dohc, dohcv, l, ohc, ohcf, ohcv, rotor.|
|16. num-of-cylinders:|eight, five, four, six, three, twelve, two.|
|17. engine-size:|numérico desde 61 to 326.|
|18. fuel-system:|1bbl, 2bbl, 4bbl, idi, mfi, mpfi, spdi, spfi.|
|19. bore:|numérico desde 2.54 hasta 3.94.|
|20. stroke:|numérico desde 2.07 hasta 4.17.|
|21. compression-ratio:|numérico desde 7 hasta 23.|
|22. horsepower:|numérico desde 48 hasta 288.|
|23. peak-rpm:|numérico desde 4150 hasta 6600.|
|24. city-mpg:|numérico desde 13 hasta 49.|
|25. highway-mpg:|numérico desde 16 hasta 54.|
|26. price:|numérico desde 5118 hasta 45400.|



## Variable Respuesta:

La variable que se pretende predecir es el consumo por galón en ciudad (city_mpg).
Separemos en conjuntos de entrenamiento y prueba.

## Conjuntos de entrenamiento y prueba.   

Se separarán en 70% de los datos en el conjunto de entrenamiento y 30% en el conjunto de prueba, usando un muestreo aleatorio simple.   

In [18]:
print(carros.columns)

Index(['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration',
       'num-of-doors', 'body-style', 'drive-wheels', 'engine-location',
       'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-type',
       'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke',
       'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
       'highway-mpg', 'price'],
      dtype='object')


In [19]:
y = carros.pop("city-mpg")
X = carros

In [20]:
X_train, X_test,y_train, y_test = train_test_split(X, y,test_size=0.3,train_size=0.7)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(143, 25)
(62, 25)
(143,)
(62,)


## Modelos

### Árbol de regresión

Haremos un modelo lineal múltiple usando todos los predictores.

Separamos la variables numéricas de las categóricas

In [26]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(carros)
categorical_columns = categorical_columns_selector(carros)
categorical_columns
print(carros.dtypes)
print(numerical_columns)
print(categorical_columns)

symboling              int64
normalized-losses     object
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                  object
stroke                object
compression-ratio    float64
horsepower            object
peak-rpm              object
highway-mpg            int64
price                 object
dtype: object
['symboling', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-size', 'compression-ratio', 'highway-mpg']
['normalized-losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors', 'body-style', 'drive-wheels', 'engine-location', 'engine-type', '

Para las categóricas se les hace un OneHotEncoder, en este encoder para cada categoría se crea una variable dicotómica

In [ ]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns)])
    

Se hace una tubería, donde se codifican las variables categóricas, y se crea el modelo de árbol de Regresión, luego se tunea el parámetro de max__depth. Se imprime el mejor parámetro

In [ ]:
param_grid = {"max_depth": np.arange(2, 10, 1)}
pipeline = Pipeline(
    [('preprocessor',preprocessor),
     ('regressor',DecisionTreeRegressor())])
params = [{
    "regressor__max_depth": np.arange(2, 10, 1)
}]
modelo1 = GridSearchCV(pipeline, param_grid=params, n_jobs=2, cv=5,scoring="neg_root_mean_squared_error")
modelo1.fit(X_train,y_train)
modelo1.best_params_

{'regressor__max_depth': np.int64(9)}

Se predice con el mejor modelo

In [24]:
modelo1_pred = modelo1.predict(X_test)
modelo1_pred

array([29.40909091, 23.69230769, 16.        , 22.        , 23.69230769,
       19.        , 23.69230769, 23.69230769, 15.        , 27.        ,
       23.69230769, 29.40909091, 33.        , 19.        , 29.40909091,
       29.40909091, 23.69230769, 29.40909091, 24.        , 19.        ,
       24.        , 29.40909091, 28.        , 24.        , 29.40909091,
       29.40909091, 23.69230769, 23.69230769, 22.        , 16.        ,
       29.40909091, 21.        , 37.33333333, 15.        , 19.        ,
       17.        , 19.        , 23.69230769, 29.40909091, 19.        ,
       16.        , 29.40909091, 29.40909091, 17.        , 29.40909091,
       19.        , 17.        , 23.69230769, 23.69230769, 23.69230769,
       17.        , 17.        , 23.69230769, 28.        , 23.69230769,
       24.        , 29.40909091, 29.40909091, 21.        , 29.40909091,
       29.40909091, 17.        ])